In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import random


# fake_data_file.npy
file_path = '/home/rzhou/Projects/Diffusion-TS/OUTPUT/rounD_map00_interval1_seq500_nfea8_pad-300/ddpm_fake_rounD_map00_interval1_seq500_nfea8_pad-300.npy'
fake_data_norm = np.load(file_path)

df = pd.read_csv("/DATA1/rzhou/ika/multi_testcases/rounD/ori/seq500/00-01/rounD_map00_interval1_seq500_nfea8.csv", header=0)
data = df.values

# 定义一个极端的填充值
extreme_value = -300
# 替换所有为0的值
data[data == 0] = extreme_value

scaler = MinMaxScaler()
scaler = scaler.fit(data[:,1:])
seq_length=500
num_feature=8
fake_data=scaler.inverse_transform(fake_data_norm.reshape(-1, num_feature)).reshape(-1, seq_length, num_feature)

fake_data[fake_data < -200] = 0

In [13]:
fake_data[0].shape
index=6829
n, n_features = fake_data[index].shape
print(n)
print(n_features)

500
8


In [14]:
# 假设data是已经加载的数据
cur_fake_data=fake_data[index]
n, n_features = cur_fake_data.shape

In [15]:
def find_first_and_last_non_zero(x_column, y_column):
    # 生成一个标记非零元素位置的布尔数组
    non_zero_mask = (x_column != 0) & (y_column != 0)
    # 如果全部是零，则返回None
    if not np.any(non_zero_mask):
        return None, None
    # 从前往后找第一个非零值的位置
    initial_frame = np.argmax(non_zero_mask)
    # 反转数组后，再次找第一个非零值的位置，实际上是原数组中最后一个非零值的位置
    final_frame = non_zero_mask.size - 1 - np.argmax(non_zero_mask[::-1])
    return initial_frame, final_frame

In [5]:
# 初始化空的DataFrame来存储tracks的元数据
columns = ['recordingId', 'trackId', 'initialFrame', 'finalFrame', 'numFrames', 'width', 'length', 'class']
tracks_meta_df = pd.DataFrame(columns=columns)


# 遍历每个track
for track_id in range(n_features // 2):
    x_column = cur_fake_data[:, track_id*2]
    y_column = cur_fake_data[:, track_id*2 + 1]
    
    initial_frame, final_frame = find_first_and_last_non_zero(x_column, y_column)
    
    # 构造这个track的元数据行
    track_meta = {
        'recordingId': 'fake_0',
        'trackId': track_id,
        'initialFrame': initial_frame,
        'finalFrame': final_frame,
        'numFrames': final_frame - initial_frame + 1,
        'width': 2,
        'length': 4,
        'class': 'car'
    }

    track_meta_df = pd.DataFrame([track_meta])
    tracks_meta_df = pd.concat([tracks_meta_df, track_meta_df], ignore_index=True)

    

# 保存到CSV文件
tracks_meta_df.to_csv('/home/rzhou/Projects/Diffusion-TS/Data/ika_format_csv/fake_00_tracksMeta.csv', index=False)



In [11]:
# 初始化存储轨迹数据的DataFrame
columns = ['recordingId', 'trackId', 'frame', 'trackLifetime', 'xCenter', 'yCenter',
           'heading', 'width', 'length', 'xVelocity', 'yVelocity',
           'xAcceleration', 'yAcceleration', 'lonVelocity', 'latVelocity',
           'lonAcceleration', 'latAcceleration']
tracks_df = pd.DataFrame(columns=columns)

delta_time = 0.04  # 假定每帧间隔时间，单位秒

# 生成轨迹数据
for track_id in range(n_features // 2):
    x_column = cur_fake_data[:, track_id * 2]
    y_column = cur_fake_data[:, track_id * 2 + 1]
    non_zero_indices = np.where((x_column != 0) & (y_column != 0))[0]
    if len(non_zero_indices) == 0:
        continue  # 如果这个track全是零，则跳过
    for frame in range(n):
        track_lifetime = frame - non_zero_indices[0] if frame >= non_zero_indices[0] else 0
        x_velocity = (x_column[frame+1] - x_column[frame]) / delta_time if frame < n - 1 else (x_column[frame] - x_column[frame-1]) / delta_time
        y_velocity = (y_column[frame+1] - y_column[frame]) / delta_time if frame < n - 1 else (y_column[frame] - y_column[frame-1]) / delta_time
        
        track_data = {
            'recordingId': 'fake_0',
            'trackId': track_id,
            'frame': frame,
            'trackLifetime': track_lifetime,
            'xCenter': x_column[frame],
            'yCenter': y_column[frame],
            'heading': 100,
            'width': 2,
            'length': 4,
            'xVelocity': x_velocity,
            'yVelocity': y_velocity,
            'xAcceleration': 0,
            'yAcceleration': 0,
            'lonVelocity': 0,
            'latVelocity': 0,
            'lonAcceleration': 0,
            'latAcceleration': 0
        }
        tracks_df = pd.concat([tracks_df, pd.DataFrame([track_data])], ignore_index=True)

# 保存到CSV文件
tracks_df.to_csv(f'./ika_format_csv/fake_00_{index}_tracks.csv', index=False)


/tmp/ipykernel_808319/3671262749.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tracks_df = pd.concat([tracks_df, pd.DataFrame([track_data])], ignore_index=True)


In [17]:
# 初始化两个空的DataFrame来分别存储tracks的元数据和tracks的详细信息
tracks_meta_columns = ['recordingId', 'trackId', 'initialFrame', 'finalFrame', 'numFrames', 'width', 'length', 'class']
tracks_columns = [
    'recordingId', 'trackId', 'frame', 'trackLifetime',
    'xCenter', 'yCenter', 'heading', 'width', 'length',
    'xVelocity', 'yVelocity', 'xAcceleration', 'yAcceleration',
    'lonVelocity', 'latVelocity', 'lonAcceleration', 'latAcceleration'
]

tracks_meta_df = pd.DataFrame(columns=tracks_meta_columns)
tracks_df = pd.DataFrame(columns=tracks_columns)

delta_time = 0.04  # 假设的每帧间隔时间，单位秒
recordingId = 'fake_0'
frame_rate = 25  # 假设的帧率

# 遍历每个track
for track_id in range(n_features // 2):
    x_column = cur_fake_data[:, track_id * 2]
    y_column = cur_fake_data[:, track_id * 2 + 1]
    initial_frame, final_frame = find_first_and_last_non_zero(x_column, y_column)
    
    if initial_frame is None or final_frame is None:
        continue  # 如果这个track全是零，则跳过

    # 构造tracksMeta的元数据行
    track_meta = {
        'recordingId': recordingId,
        'trackId': track_id,
        'initialFrame': initial_frame,
        'finalFrame': final_frame,
        'numFrames': final_frame - initial_frame + 1,
        'width': 2,
        'length': 4,
        'class': 'car'
    }
    tracks_meta_df = pd.concat([tracks_meta_df, pd.DataFrame([track_meta])], ignore_index=True)

    # 添加到tracks_df
    for frame in range(initial_frame, final_frame + 1):
        x_velocity = (x_column[min(frame + 1, final_frame)] - x_column[frame]) / delta_time
        y_velocity = (y_column[min(frame + 1, final_frame)] - y_column[frame]) / delta_time

        track_data = {
            'recordingId': recordingId,
            'trackId': track_id,
            'frame': frame,
            'trackLifetime': frame - initial_frame,
            'xCenter': x_column[frame],
            'yCenter': y_column[frame],
            'heading': 100,
            'width': 2,
            'length': 4,
            'xVelocity': x_velocity,
            'yVelocity': y_velocity,
            'xAcceleration': 0,
            'yAcceleration': 0,
            'lonVelocity': 0,
            'latVelocity': 0,
            'lonAcceleration': 0,
            'latAcceleration': 0
        }
        tracks_df = pd.concat([tracks_df, pd.DataFrame([track_data])], ignore_index=True)

# 定义文件路径
tracks_meta_csv_path = f'/home/rzhou/Projects/Diffusion-TS/Data/ika_format_csv/fake_00_{index}_tracksMeta.csv'
tracks_csv_path = f'/home/rzhou/Projects/Diffusion-TS/Data/ika_format_csv/fake_00_{index}_tracks.csv'

# 清空文件内容
open(tracks_meta_csv_path, 'w').close()
open(tracks_csv_path, 'w').close()

# 现在文件已被清空，可以安全地写入新内容
tracks_meta_df.to_csv(tracks_meta_csv_path, index=False)
tracks_df.to_csv(tracks_csv_path, index=False)



/tmp/ipykernel_808319/2299210175.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tracks_df = pd.concat([tracks_df, pd.DataFrame([track_data])], ignore_index=True)
